In [1]:
from datetime import datetime
from datetime import timedelta
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import yfinance as yf
import os
import pandas as pd
import numpy as np

end_date = datetime.today()
DATA_DIR = "./DATA"
DOW_ticker = 'djia'
NASDAQ_ticker = 'comp'
SP500_ticker = 'spx'
# need_to_update_file_name = "Dow_daily.csv"
# need_to_update_file_name = "SP500_daily.csv"
need_to_update_file_name = "Nasdaq_daily.csv"
need_to_update_ticker = NASDAQ_ticker

In [2]:
# read from local file

def read_local_file(file_path):
    dataframe = pd.read_csv(os.path.join(DATA_DIR, file_path))
    dataframe = dataframe.dropna(how='all')
    print(dataframe)
    dataframe = dataframe.set_index('Date')
    sss = dataframe[-1:]['Close'].idxmin()
    print(sss)
    try:
        latest_date = datetime.strptime(dataframe[-1:]['Close'].idxmin(), '%Y-%m-%d')
    except ValueError:
        latest_date = datetime.strptime(dataframe[-1:]['Close'].idxmin(), '%Y/%m/%d')
    tomorrow_date = latest_date + timedelta(1)
    return tomorrow_date, dataframe

In [3]:
tomorrow_date, original_df = read_local_file(need_to_update_file_name)
print("tomorrow_date -> ", tomorrow_date)
print("original_df.tail", original_df.tail())

            Date   Adj Close         Close          High           Low  \
0     1990-01-02  459.299988    459.299988    459.299988    452.700012   
1     1990-01-03  460.899994    460.899994    461.600006    460.000000   
2     1990-01-04  459.399994    459.399994    460.799988    456.899994   
3     1990-01-05  458.200012    458.200012    459.399994    457.799988   
4     1990-01-08  458.700012    458.700012    458.700012    456.500000   
...          ...         ...           ...           ...           ...   
7865  2021-03-22         NaN  13377.540000  13455.650000  13278.780000   
7866  2021-03-23         NaN  13227.700000  13405.150000  13202.430000   
7867  2021-03-24         NaN  12961.890000  13292.920000  12961.350000   
7868  2021-03-25         NaN  12977.680000  13021.860000  12786.810000   
7869  2021-03-26         NaN  13138.720000  13149.550000  12878.720000   

              Open       Volume  
0       452.899994  110720000.0  
1       461.100006  152660000.0  
2       4

In [4]:
# crawl from the web
def crawl_from_web(ticker, start_date, end_date):
    start_date_str = start_date.strftime("%m/%d/%Y")
    end_date_str = end_date.strftime("%m/%d/%Y")
    market_watch_url = 'https://www.marketwatch.com/investing/index/{}/download-data?startDate={}&endDate={}'
    market_watch_url = market_watch_url.format(ticker, start_date_str, end_date_str)
    print("url -> ",  market_watch_url)
    req = Request(url=market_watch_url,headers={'user-agent': 'my-app/0.0.1'}) 
    response = urlopen(req)    
    html = BeautifulSoup(response)
    content = html.find_all("div", {"class": "overflow--table"})[0]
    rows = content.findAll('tr')[1:]
    collected_data = []
    for i, row in enumerate(rows):
        # [<div class="cell__content fixed--cell u-secondary">03/26/2021</div>, <div class="cell__content u-secondary">03/26/2021</div>, <div class="cell__content">41,086.51</div>, <div class="cell__content">41,637.09</div>, <div class="cell__content">40,964.56</div>, <div class="cell__content">41,604.09</div>]
        # [Open, High, Low, Close]
        date_str = row.findAll('div')[0].text
        temp_date = datetime.strptime(date_str, "%m/%d/%Y")
        if temp_date >= start_date:
            temp_row = []
            for j, cell in enumerate(row.findAll('div')):
                if j == 0:
                    temp_row.append(temp_date.strftime('%Y-%m-%d'))
                if j > 1:
                    temp_row.append(float(cell.text.replace(',', '').strip()))
            collected_data.append(temp_row)
        
    
    dataframe = pd.DataFrame(collected_data, columns = ['Date', 'Open', 'High', 'Low', 'Close'])
    dataframe = dataframe.set_index('Date')
    dataframe = dataframe.sort_index()
    return dataframe


In [5]:
collected_df = crawl_from_web(need_to_update_ticker, tomorrow_date, end_date)

url ->  https://www.marketwatch.com/investing/index/comp/download-data?startDate=03/27/2021&endDate=03/30/2021


In [6]:
collected_df

,Open,High,Low,Close
Date,,,,
2021-03-29,13103.97,13143.41,12968.16,13059.65


In [7]:
original_df

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
1990-01-02,459.299988,459.299988,459.299988,452.700012,452.899994,110720000.0
1990-01-03,460.899994,460.899994,461.600006,460.000000,461.100006,152660000.0
1990-01-04,459.399994,459.399994,460.799988,456.899994,460.399994,147950000.0
1990-01-05,458.200012,458.200012,459.399994,457.799988,457.899994,137230000.0
1990-01-08,458.700012,458.700012,458.700012,456.500000,457.100006,115500000.0
...,...,...,...,...,...,...
2021-03-22,NaN,13377.540000,13455.650000,13278.780000,13278.780000,NaN
2021-03-23,NaN,13227.700000,13405.150000,13202.430000,13381.440000,NaN
2021-03-24,NaN,12961.890000,13292.920000,12961.350000,13289.240000,NaN


In [8]:
frames = [original_df, collected_df]
result = pd.concat(frames, sort=True)

In [9]:
result

,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
1990-01-02,459.299988,459.299988,459.299988,452.700012,452.899994,110720000.0
1990-01-03,460.899994,460.899994,461.600006,460.000000,461.100006,152660000.0
1990-01-04,459.399994,459.399994,460.799988,456.899994,460.399994,147950000.0
1990-01-05,458.200012,458.200012,459.399994,457.799988,457.899994,137230000.0
1990-01-08,458.700012,458.700012,458.700012,456.500000,457.100006,115500000.0
...,...,...,...,...,...,...
2021-03-23,NaN,13227.700000,13405.150000,13202.430000,13381.440000,NaN
2021-03-24,NaN,12961.890000,13292.920000,12961.350000,13289.240000,NaN
2021-03-25,NaN,12977.680000,13021.860000,12786.810000,12844.580000,NaN


In [10]:
result.to_csv(os.path.join(DATA_DIR, need_to_update_file_name))